Notebook for evaluating models.

Requires you to have in your drive:
- the predictions generated by your model
- the target sentences they should resemble
- for some metrics the source sentences are required

----

WHAT YOU NEED TO (MAYBE) CHANGE

In section 0.
- you can add more metrics. Go to https://huggingface.co/docs/datasets/metrics to see what metrics huggingface has.

In section 1.
 - Modify the file_path to match where your model is stored
 - You may need to modify the dataframe column names

In section 2. 
 - You might need to change the parameters, such as 'prefix' for cleaning the predictions and source data

# 0 - Load imports

In [ ]:
!pip install --upgrade pip -q  # ensures that pip is current 

     |████████████████████████████████| 2.1 MB 5.2 MB/s 


## 0.1 Main imports (including BLEU, METEOR)

In [ ]:
!pip install -U nltk -q
!pip install datasets -q
!pip install -U PyYaml==5.1

     |████████████████████████████████| 362 kB 5.4 MB/s 
     |████████████████████████████████| 140 kB 67.6 MB/s 
     |████████████████████████████████| 212 kB 59.3 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 144 kB 64.9 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 5.3 MB/s 
  

In [ ]:
# Util
import os
import pandas as pd
import numpy as np
import re
from google.colab import drive

# Load metrics
from datasets import load_metric, load
meteor = load_metric("meteor")
# glue = load_metric("glue", "mrpc")
bleu = load_metric("bleu")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


## 0.2 Load BLEURT

https://github.com/google-research/bleurt#readme

In [ ]:
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .

Cloning into 'bleurt'...
remote: Enumerating objects: 116, done.
^C
[Errno 2] No such file or directory: 'bleurt'
/content
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

--2022-06-12 12:27:09--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.16, 172.217.0.48, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2140294207 (2.0G) [application/octet-stream]
Saving to: ‘BLEURT-20.zip’

BLEURT-20.zip       100%[===================>]   1.99G   162MB/s    in 12s     

2022-06-12 12:27:20 (176 MB/s) - ‘BLEURT-20.zip’ saved [2140294207/2140294207]

--2022-06-12 12:27:20--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2022-06-12 12:27:20--
Total wall clock time: 12s
Downloaded: 1 files, 2.0G in 12s (176 MB/s)
Archive:  BLEURT-20.zip
   creating: BLEURT-20/
  inflating: BLEURT-20/bert_config.json  
  inflating: BLEURT-20/saved_model.pb  
   creating: BLEURT-20/variables/
  inflating: BL

## 0.3 Load COMET (not working)


In [ ]:
# Install Comet (doesn't always work - often gets strange dependency issues, or just denies that it is installed)

!pip install unbabel-comet -q
comet = load_metric("comet")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 59.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 re

wmt20-comet-da.tar.gz: 1.79GB [00:47, 37.8MB/s]                            
Extracting /root/.cache/torch/unbabel_comet/wmt20-comet-da.tar.gz
Extracted /root/.cache/torch/unbabel_comet/wmt20-comet-da.tar.gz
Created a temporary directory at /tmp/tmpsqp10jis
Writing /tmp/tmpsqp10jis/_remote_module_non_sriptable.py


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


# 1 - Get the data 
Predictions, Sources & References to Compare

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !! Modify this (or just add your own) based on the location of your files in your drive

# file_path = "file path here"

In [ ]:
# # if taking the original data file

# data_cleaned = pd.read_csv(file_path+"data_cleaned.csv")

In [ ]:
# If loading in prediction data

# Load in the two files
predictions_file = pd.read_csv(file_path+"predictions_IOB.csv")

#Isolate the columns we need
# !! You might need to change the column names based on your files
predictions = predictions_file['Generated Text']
references = predictions_file['Actual Text']
sources = predictions_file['Input']
# input_IOBs = predictions_file['IOB_input']
# target_IOBs = predictions_file['IOB_target']
input_IOBs = predictions_file['Source IOB']
target_IOBs = predictions_file['Target IOB']

# # Load in the files (models Ryan)
# predictions_file = pd.read_csv(file_path+"predictions_IOB.csv")

# #Isolate the columns we need
# # !! You might need to change the column names based on your files
# predictions = predictions_file['Generated Text']
# references = predictions_file['Actual Text']
# sources = predictions_file['Input']
# input_IOBs = predictions_file['Source IOB']
# target_IOBs = predictions_file['Target IOB']

# 2 - Process and Clean the Data

## 2.1 Basic cleaning (necessary)

Check the data to see what exactly needs to be removed, and if punctuation needs to be separated for consistency across each of the generated, target and sources

In [ ]:
def cleanPredictions(predictions, unwanted=[":"], space_punct=True):
  ''' Function to remove the unwanted characters at the beginning of a sentence.
      Sometimes the predictions still have ":" at the beginning
      Also has an option to space out punctuation if that does not match here for some reason
  '''

  if space_punct:
    predictions = predictions.str.replace(",", " ,")
    predictions = predictions.str.replace(".", " .")
    predictions = predictions.str.replace("'", " '")
    predictions = predictions.str.replace("  ", " ")


  predictions = predictions.str.replace('"', '')
  for i in unwanted:
    predictions = predictions.str.replace(i, "")
  return predictions

def cleanSources(sources, prefix, space_punct=True, suffix=None):
  '''Function to remove the prefix from the source sentence
  For example, if you had the prefix "paraphrase:" then you should add that as the prefix parameter.
  '''
  if space_punct:
    sources = sources.str.replace(",", " ,")
    sources = sources.str.replace(".", " .")
    sources = sources.str.replace("'", " '")
    sources = sources.str.replace("  ", " ")


  sources = sources.str.replace('"', '')
  # Removes any remainder of a prefix that might have stuck
  for p in prefix:
    sources = sources.str.replace(p, "")
  
  # Removes the idiom and everything following it
  if suffix is not None:
    for i in range(len(suffix)):
      sources[i] = sources[i].split(suffix, 1)[0]
  return sources

In [ ]:
sources[0]

"Let 's give her the benefit of the doubt and assume that she is right . "

In [ ]:
predictions[0]

"Let 's doubt her and assume that she is right ."

In [ ]:
references[0]

"Let 's regard her as innocent and assume that she is right ."

In [ ]:
input_IOBs[0]

'O O B O I I I I I O O O O O O O'

In [ ]:
target_IOBs[0]

'O O O O O O O O O O O O O O O O O O O O O O O'

In [ ]:
# !! Specify here what are the things that need to be removed!

# predictions = cleanPredictions(predictions, unwanted=[": ", "phrased: ", "phrased"])
predictions = cleanPredictions(predictions, unwanted=["*"])
references = cleanPredictions(references, unwanted=["*"])

# Following line is required for COMET, which does not work right now
# sources = cleanSources(sources, prefix=["idiom paraphrase: "], suffix="idiom: ")
sources = cleanSources(sources, prefix=["id_par sentence: ", "*"], suffix="idiom: ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: FutureWarning: The default value of regex will change from True to False in a 

## 2.2 FUNC: Isolate idioms & corresponding phrases (necessary for IdiomScore)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

extras = []
removals = []

for e in extras:
  sw_nltk.append(e)

for r in removals:
  sw_nltk.remove(r)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sw_nltk

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def phrase_from_sentence_IOB(IOB, sentence):
  phrase = []
  for i in range(0, len(sentence)):
    if IOB[i] == "B" or IOB[i] == "I":
      phrase.append(sentence[i])
  return ' '.join(phrase)

def phrase_from_genSentence_IOB(IOB_source, IOB_target, source, target, generated, margin=1, verbose=False, remove_stopwords=False):
  # Find the source phrase using IOB tags
  # Find the same phrase in generated, with margin of 0 or 1 words either side
  # Also get the target phrase, for human comparison
  # Return all three phrases
  
  # TARGET PHRASE
  targetPhrase = []
  # Get the target phrase (not used for scoring, but nice to have in the dataframe)
  for i in range(len(IOB_target)):
    try:
      if IOB_target[i] == "B":
        targetPhrase.append(target[i])
      if IOB_target[i] == "I":
        targetPhrase.append(target[i])
    except Exception as e:
      if verbose:
        print(e, f" - target, i = {i}")
  # Turn the list into a string, separated by spaces
  targetPhrase = ' '.join(targetPhrase)

  # SOURCE PHRASE
  sourcePhrase = []
  generatedPhrase = []

  # Find last idiom (for margin)
  last_idiom = 0
  if margin is not None:
    for i in range(len(IOB_source)):
      if IOB_source[i] == "I":
        last_idiom = i

  # Get the source idiom locations and create source and generated subphrases
  for i in range(len(IOB_source)):
    try:
      # add the starting idiom word (marked with "B" in the IOB tags)
      if IOB_source[i] == "B":
        # if a margin is requested, get those words from the left of the starting idiom word
        if margin is not None:
          for m in range(margin):
            generatedPhrase.append(generated[i-(m+1)])
        # add corresponding word at location [i] from each sentence to respective subphrase list
        sourcePhrase.append(source[i])
        generatedPhrase.append(generated[i])
      # do the same for each word marked as "I" (non starting idiom word)
      if IOB_source[i] == "I":
        sourcePhrase.append(source[i])
        generatedPhrase.append(generated[i])
        # if a margin is requested, get those words from the right of the final idiom word
        if i == last_idiom and margin is not None:
          for m in range(margin):
            generatedPhrase.append(generated[i+(m+1)])
    except Exception as e:
      if verbose:
        print(e, f" - source, i = {i}")
  
  # Remove stopwords if requested
  if remove_stopwords:
    # loop over each word in the nltk stopword list, and check if its in our generated subphrases
    for sw in sw_nltk:
      for s in sourcePhrase:
        if s == sw:
          sourcePhrase.remove(s)
      for g in generatedPhrase:
        if g == sw:
          generatedPhrase.remove(g)
  # Turn the lists into strings, separated by spaces
  sourcePhrase = ' '.join(sourcePhrase)
  generatedPhrase = ' '.join(generatedPhrase)

  return sourcePhrase, targetPhrase, generatedPhrase

## 2.3 FUNC: Turn database of strings to a list of lists of strings (necessary for BLEU, IdiomScore)

In [ ]:
def dfStringsToLists(sr, preds=True):
  ''' Function (created for Bleu metric) that turns a database of strings, into a list of lists of strings, for each word
      For predictions:
      i.e. if df[0] = 'This is a sentence', then it returns ['This','is','a','sentence']
      References for bleu require another wrapping of lists:
      i.e. if df[0] = 'This is a sentence', then it returns [['This','is','a','sentence']]

      Parameters:
      sr:     pd Series to turn into a frame, then isolate then values
      preds:  Boolean value, determines whether to add another list wrapper to element
  '''
  df = sr.copy().to_frame()
  df = df.values[:,0].tolist()

  for i in range(len(df)):
    try:
      df[i] = df[i].split(" ")
    except:
      df[i] = [""]
    if not preds:     # references need to be encased in another list
      df[i] = [df[i]]
  

  return df

# 3 - Get the Scores

## 3.1 Balanced Idiom Paraphrase Metric

### Create the dataframe of phrases

In [ ]:
# PARAMETERS:
margin = 1 # determines how many extea words to either side of discovered subphrase should be taken.
remove_stopwords = True

# create a new dataframe to store the values in
df_phrases = pd.DataFrame(columns=['Source Phrase', 'Prediction Phrase', 'Reference Phrase', 'Source Full',  'Prediction Full', 'Reference Full'])
# loop over the generated, input and target sentences & IOB tags, split them on whitespace to count the words
for j, pred in enumerate(predictions):
  pred = pred.split(' ')
  source = sources[j].split(' ')
  reference = references[j].split(' ')
  input_IOB = input_IOBs[j].split(' ')
  target_IOB = target_IOBs[j].split(' ')
  # get the subphrases from the function defined above (SEC 2.2)
  sourcePhrase, referencePhrase, predictionPhrase = phrase_from_genSentence_IOB(input_IOB, target_IOB, source, reference, pred, margin=margin, remove_stopwords=remove_stopwords)
  df_phrases.loc[j] = {'Source Phrase': sourcePhrase, 'Prediction Phrase': predictionPhrase, 'Reference Phrase': referencePhrase, 'Source Full': sources[j], 'Prediction Full': predictions[j], 'Reference Full': references[j]}

In [ ]:
df_phrases.iloc[0]

Source Phrase                                       give benefit doubt
Prediction Phrase                                's doubt assume right
Reference Phrase                                    regard as innocent
Source Full          Let 's give her the benefit of the doubt and a...
Prediction Full        Let 's doubt her and assume that she is right .
Reference Full       Let 's regard her as innocent and assume that ...
Name: 0, dtype: object

In [ ]:
df_phrases

,Source Phrase,Prediction Phrase,Reference Phrase,Source Full,Prediction Full,Reference Full
0,give benefit doubt,'s doubt assume right,regard as innocent,Let 's give her the benefit of the doubt and a...,Let 's doubt her and assume that she is right .,Let 's regard her as innocent and assume that ...
1,ballpark figure,estimated cost,rough estimate,The management was given a ballpark figure at ...,The management was given an estimated cost at ...,The management was given a rough estimate at t...
2,bottom heart,.,sincere feeling,I am really happy with the new job and I mean ...,I am really happy with the new job and I mean ...,I am really happy with the new job and I mean ...
3,turning point,The boiling point,significant change,The turning point in the story came when the p...,The boiling point in the story came when the p...,The significant change in the story came when ...
4,goes mile,teacher,makes an effort,"When it comes to weaker students , the teacher...","When it comes to weaker students , the teacher...","When it comes to weaker students , the teacher..."
...,...,...,...,...,...,...
818,stands reason,company agrees project,reasons,The company stands to reason that the project ...,The company agrees that the project will take ...,The company reasons that the project will take...
819,,eat,all,Is the food ready yet? I am hungry and ready t...,Is the food ready yet? I am hungry and ready t...,Is the food ready yet? I am hungry and ready t...
820,fast,fast,junk,Nina loves French fries and other fast food . ...,Nina loves French fries and other fast food .,Nina loves French fries and other junk food .
821,caught,obsessed beauty,fascinated with,I was so caught up in the beauty of the art wo...,I was so obsessed with the beauty of the art w...,I was so fascinated with the beauty of the art...


### Do the actual evaluation

Setting up BLEURT

In [ ]:
# (!) Only if using BLEURT instead of BLEU (not really recommended)

from bleurt import score
checkpoint = "bleurt/test_checkpoint"
scorer = score.BleurtScorer(checkpoint)

ModuleNotFoundError: ignored

In [ ]:
# Negative score of syntactic similarity of sourcePhrase with generatedPhrase (BLEU) (beta)
# Positive socre of semantic similarity of generatedSentence with targetSentence (METEOR) (alpha)

# Score version one = alpha - beta

# TODO: Make this function run for each sentence, store the scores for each in new column

def balancedIdiomParaphraseMetric(sourcePhrases, 
                                  generatedPhrases, 
                                  generatedSentences, 
                                  targetSentences,
                                  useBLEU = True,
                                  useBLEURT = False, 
                                  alpha = 1, 
                                  beta = 1):
  '''
  sourcePhrases: df of idiomatic subphrase isolated from source sentence (FUNC 2.2)
  generatedPhrases: df of paraphrased idiomatic subphrases isolated from generated sentence (FUNC 2.2)
  generatedSentences: df of the predictions the model generated
  targetSentences: df of the reference sentences
  useBLEU: boolean to use BLEU for syntactic similarity of subphrases (instead of BLEURT)
  useBLEURT: boolean to use BLEURT for syntactic similarity of subphrases (instead of BLEU)
  alpha: weight to apply to the meteor (semantic similarity) score (not currently used)
  beta: weight to apply to the bleu (syntactic (dis)similarity) score (not currently used)
  '''
  assert useBLEU or useBLEURT, f"Please set either useBLEU or useBLEURT to True - Currently set to: useBLEU == {useBLEU}, useBLEURT == {useBLEURT}"
  assert (not useBLEU) or (not useBLEURT), f"Please set either useBLEU or useBLEURT to True - Currently set to: useBLEU == {useBLEU}, useBLEURT == {useBLEURT}"
  
  # Score 1 = semantic similarity of generated and target sentences
  meteorScore = meteor.compute(predictions=generatedSentences, references=targetSentences)
  semanticSimScore = alpha * meteorScore['meteor']

  # Score 2 = syntactic (dis)similarity of source and target idiom subphrases

  # First change the strings to lists (necessary for BLEU)
  syntacticDissimScore = 0
  if useBLEU:
    genPhraseAsLists = dfStringsToLists(generatedPhrases, preds=True)
    sourcePhrasesAsLists = dfStringsToLists(sourcePhrases, preds=False)

    bleuScore = bleu.compute(predictions=genPhraseAsLists, references=sourcePhrasesAsLists)
    syntacticDissimScore = beta * (sum(bleuScore['precisions']) / len(bleuScore['precisions']))
  else: # using BLEURT
    scores = scorer.score(references=sourcePhrases, candidates=generatedPhrases)
    syntacticDissimScore = sum(scores) / len(scores)

  return { 'semantic': semanticSimScore, 'syntactic': syntacticDissimScore, 'idiom_score': semanticSimScore - syntacticDissimScore}
  

In [ ]:
# Choose ONE of the following
useBLEU = True
useBLEURT = False

In [ ]:
idiomScore = balancedIdiomParaphraseMetric(df_phrases["Source Phrase"], df_phrases["Prediction Phrase"], predictions, references, useBLEU=useBLEU, useBLEURT=useBLEURT)

print(idiomScore)

{'semantic': 0.8518524816291221, 'syntactic': 0.02804203843956168, 'idiom_score': 0.8238104431895604}


### Get the score for each individual sentence, store it in a csv

In [ ]:
df_scores = pd.DataFrame(columns=['Semantic Score', 'Syntactic Score', 'Idiom Score'])

for i, _ in enumerate(df_phrases['Prediction Phrase']):
  # isolating the specific row
  phrase_i = df_phrases.loc[i]
  # isolating the sentences, then putting them in [] to make them be elements in an array (necessary for metrics)
  idiomScore = balancedIdiomParaphraseMetric(pd.Series(phrase_i["Source Phrase"]), pd.Series(phrase_i["Prediction Phrase"]), pd.Series(predictions[i]), pd.Series(references[i]),  useBLEU=useBLEU, useBLEURT=useBLEURT)
  # adding to a new data frame
  df_scores.loc[i] = {'Semantic Score': str(idiomScore['semantic']), 'Syntactic Score': str(idiomScore['syntactic']), 'Idiom Score': str(idiomScore['idiom_score']) }

# joining both dataframes to a new one
df_phrases_scores = df_phrases.join(df_scores)

In [ ]:
df_phrases_scores.head()

,Source Phrase,Prediction Phrase,Reference Phrase,Source Full,Prediction Full,Reference Full,Semantic Score,Syntactic Score,Idiom Score
0,give benefit doubt,give benefit prove,,I don 't believe that he didn 't take the mone...,I don 't believe that he didn 't take the mone...,I don 't believe that he didn 't take the mone...,0.9330026360786549,0.29166666666666663,0.6413359694119882
1,ballpark amount,father sum,an estimate,She manages to give her father a ballpark amou...,She manages to give her father a sum that she ...,She manages to give her father an estimate tha...,0.8650887573964497,0.0,0.8650887573964497
2,bottom heart,feelings .,genuinely,It was really good to have you here and I woul...,It was really good to have you here and I woul...,It was really good to have you here and I woul...,0.7748538011695907,0.0,0.7748538011695907
3,turning points,many positive experiences,changes,I have seen many turning points in my life and...,I have seen many positive experiences in my li...,I have seen many changes in my life and don 't...,0.9558986277113768,0.0,0.9558986277113768
4,go extra mile,They make every effort,special efforts,I love staying at that hotel and They go the e...,I love staying at that hotel and They make eve...,I love staying at that hotel and They make spe...,0.9160383357558138,0.0,0.9160383357558138


In [ ]:
# Save dataframe of individual scores
df_phrases_scores.to_csv(os.path.join(file_path+"idiom_scores_noSW.csv"))

# Save text file of average score of full data set
average_idiom_score_txt = open(file_path+"average_idiom_score_noSW.txt", "w")
average_idiom_score_txt.write(str(idiomScore))
average_idiom_score_txt.close()

### Correctness Check

Checking for correctness of the evaluation metric.

METEOR: Comparing reference sentences against themselves (expecting 100%)

BLEU: Comparing target subphrases against source subphrases (expecting low, ideally 0%)

In [ ]:
perfectIdiomScore = balancedIdiomParaphraseMetric(df_phrases["Source Phrase"], df_phrases["Reference Phrase"], references, references,  useBLEU=useBLEU, useBLEURT=useBLEURT)

print(perfectIdiomScore)

{'semantic': 0.9998391577628789, 'syntactic': 0.002037392138063279, 'idiom_score': 0.9978017656248156}


## 3.2 Other metrics: METEOR, BLEU, COMET, GLUE, ...

In [ ]:
def evaluate_predictions(predictions, references, 
                         sources=None,
                         eval_meteor=True, 
                         eval_bleu=True,
                         eval_comet=False,
                         eval_glue=False,
                         verbose=True):
  ''' Function to evaluate scores based on predictions, reference and source sentences.
  Can be expanded to add additional metrics.
  Parameters:
    predictions:  Predictions generated by the model
    references:   Target outputs to compare predictions to
    sources:      Source sentences, required for comet
    eval_meteor:  Boolean to specify if meteor metric should be evaluated
    eval_bleu:    Boolean to specify if bleu metric should be evaluated
    eval_comet:   Boolean to specify if comet metric should be evaluated (Doesn't work right now)
    eval_glue:    Boolean to specify if glue metric should be evaluated (Doesn't work right now)
    verbose:      Whether or not to print the sentences
  '''
  if eval_comet:
    assert sources is not None, "If using comet, source sentences must be added."

  evaluations = []

  # Evaluate METEOR if specified
  if eval_meteor:
    meteor_score = meteor.compute(predictions=predictions, references=references)
    evaluations.append(('meteor', meteor_score))
    if verbose:
      print(f"Meteor score = {meteor_score}")

  # Evaluate BLEU if specified
  if eval_bleu:
    preds = dfStringsToLists(predictions, preds=True)
    refs = dfStringsToLists(references, preds=False)
    # print(preds)
    # print(refs)
    bleu_score = bleu.compute(predictions=preds, references=refs)
    evaluations.append(('bleu', bleu_score))
    if verbose:
      print(f"Bleu score = {bleu_score}")
      
  # Evaluate COMET if specified
  if eval_comet:
    comet_score = comet.compute(sources=sources, predictions=predictions, references=references)
    evaluations.append(('comet', comet_score))
    if verbose:
      print(f"Comet score = {comet_score}")
    
  # Evaluate GLUE if specified
  if eval_glue:
    glue_score = glue.compute(predictions=preds, references=refs)
    evaluations.append(('glue', glue_score))
    if verbose:
      print(f"Glue score = {glue_score}")

  return evaluations

In [ ]:
eval_meteor = True
eval_bleu   = True
eval_comet  = True #doesn't currently work
eval_glue   = False #doesn't currently work

evals = evaluate_predictions(predictions, references, sources, eval_meteor=eval_meteor, eval_bleu=eval_bleu, eval_comet=eval_comet, eval_glue=eval_glue)

Meteor score = {'meteor': 0.8518524816291221}
Bleu score = {'bleu': 0.7677906488072811, 'precisions': [0.8775676778669569, 0.807911232797752, 0.7575061274509803, 0.7101520353114271], 'brevity_penalty': 0.9770063365827409, 'length_ratio': 0.9772666843924488, 'translation_length': 14702, 'reference_length': 15044}
